In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2 
import random
from joblib import dump
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# read data
def load_data():
    data = pd.read_excel('E:/Documents/Semester 8/TAmbun/Sidang/DBSCAN/Prabowo/Datasetprabowo.xlsx')
    return data

In [5]:
data = load_data()
df = data
df

,USER,comment
0,__uchie__,"""Jika nanti kita bertemu kembali dan setiap ka..."
1,bakti2717,Sehat selalu Pak Prabowo ..semoga menjadi peng...
2,alby09109,Mimpi
3,ichal_collins,Bismillah Anies Presiden RI 2024..
4,imansanusi1111,Hamparan kami bapa istirahat.aja Kasih pa Ganj...
...,...,...
206680,beckheribun,Ganjar dan Prabowo 😎🤗😇....Amin🙏
206681,mvnafri,Di facebook juga tempo dot com yg saya baca be...
206682,alfakirani,@tempodotco mau jadi media seperti lampu merah?
206683,zalmi_hendrico,Kalang kabut kalau pak Anis jd presiden 🤣🤣🤣


### Data Cleaning

In [6]:
# Drop attribut
df = df.drop('USER', axis=1)
df.head(5)

,comment
0,"""Jika nanti kita bertemu kembali dan setiap ka..."
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...
2,Mimpi
3,Bismillah Anies Presiden RI 2024..
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...


In [7]:
import re
import numpy as np

def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    return text

# Mengubah kolom 'comment' menjadi tipe data string
df['comment'] = df['comment'].astype(str)

# Menerapkan fungsi remove_pattern pada kolom 'comment' dan menyimpan hasilnya ke kolom 'clean_comment'
df['comment_cl1'] = np.vectorize(remove_pattern)(df['comment'], " *RT* | *@[\w]*")

# Menampilkan beberapa baris teratas dari DataFrame
df.head()

,comment,comment_cl1
0,"""Jika nanti kita bertemu kembali dan setiap ka...","""Jika nanti kita bertemu kembali dan setiap ka..."
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo ..semoga menjadi peng...
2,Mimpi,Mimpi
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024..
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat.aja Kasih pa Ganj...


In [8]:
# remove simbol/character
def remove(text):
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    return text
df['comment_cl2'] = df['comment_cl1'].apply(lambda x: remove(x))
df.head()

,comment,comment_cl1,comment_cl2
0,"""Jika nanti kita bertemu kembali dan setiap ka...","""Jika nanti kita bertemu kembali dan setiap ka...",Jika nanti kita bertemu kembali dan setiap kat...
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo semoga menjadi pengga...
2,Mimpi,Mimpi,Mimpi
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...


In [9]:
# remove
def remove(tweet):
    #remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    #remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    #remove hastags, only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)

    return tweet
df['comment_cl3'] = df['comment_cl2'].apply(lambda x: remove(x))
df.head()

,comment,comment_cl1,comment_cl2,comment_cl3
0,"""Jika nanti kita bertemu kembali dan setiap ka...","""Jika nanti kita bertemu kembali dan setiap ka...",Jika nanti kita bertemu kembali dan setiap kat...,Jika nanti kita bertemu kembali dan setiap kat...
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo semoga menjadi pengga...,Sehat selalu Pak Prabowo semoga menjadi pengga...
2,Mimpi,Mimpi,Mimpi,Mimpi
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024,Bismillah Anies Presiden RI
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...


### Teks Preprocessing

##### Lowercasing

In [10]:
# lowercasing
df.drop(["comment_cl1"], axis=1, inplace=True) 

df['Text_lower'] = df['comment_cl3'].str.lower()
df

,comment,comment_cl2,comment_cl3,Text_lower
0,"""Jika nanti kita bertemu kembali dan setiap ka...",Jika nanti kita bertemu kembali dan setiap kat...,Jika nanti kita bertemu kembali dan setiap kat...,jika nanti kita bertemu kembali dan setiap kat...
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo semoga menjadi pengga...,Sehat selalu Pak Prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...
2,Mimpi,Mimpi,Mimpi,mimpi
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024,Bismillah Anies Presiden RI,bismillah anies presiden ri
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,hamparan kami bapa istirahat aja kasih pa ganj...
...,...,...,...,...
206680,Ganjar dan Prabowo 😎🤗😇....Amin🙏,Ganjar dan Prabowo Amin,Ganjar dan Prabowo Amin,ganjar dan prabowo amin
206681,Di facebook juga tempo dot com yg saya baca be...,Di facebook juga tempo dot com yg saya baca be...,Di facebook juga tempo dot com yg saya baca be...,di facebook juga tempo dot com yg saya baca be...
206682,@tempodotco mau jadi media seperti lampu merah?,mau jadi media seperti lampu merah,mau jadi media seperti lampu merah,mau jadi media seperti lampu merah
206683,Kalang kabut kalau pak Anis jd presiden 🤣🤣🤣,Kalang kabut kalau pak Anis jd presiden,Kalang kabut kalau pak Anis jd presiden,kalang kabut kalau pak anis jd presiden


#### Stopword Removal

In [11]:
# Menambah kamus stopword yang belum ada di python sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

stop_factory = StopWordRemoverFactory().get_stop_words()
more_stopwords = [
    'yg', 'utk', 'cuman', 'deh', 'Btw', 'tapi', 'gua', 'gue', 'lo', 'lu',
    'kalo', 'trs', 'jd', 'nih', 'ntar', 'nya', 'lg', 'gk', 'ecusli', 'dpt',
    'dr', 'kpn', 'kok', 'kyk', 'donk', 'yah', 'u', 'ya', 'ga', 'km', 'eh',
    'sih', 'eh', 'bang', 'br', 'kyk', 'rp', 'jt', 'kan', 'gpp', 'sm', 'usah',
    'mas', 'sob', 'thx', 'ato', 'jg', 'gw', 'wkwk', 'mak', 'haha', 'iy', 'k',
    'tp', 'haha', 'dg', 'dri', 'duh', 'ye', 'wkwkwk', 'syg', 'btw',
    'nerjemahan', 'gaes', 'guys', 'moga', 'kmrn', 'nemu', 'yukkk',
    'wkwkw', 'klas', 'iu', 'ew', 'lho', 'sbnry', 'org', 'gtu', 'bwt',
    'klrga', 'clau', 'lbh', 'cpet', 'ku', 'uke', 'mba', 'mas', 'sdh', 'kmrn',
    'oi', 'spt', 'dlm', 'bs', 'krn', 'jgn', 'sapa', 'spt', 'sh', 'wakakaka',
    'sihhh', 'hehe', 'ih', 'dgn', 'la', 'kl', 'ttg', 'mana', 'kmna', 'kmn',
    'tdk', 'tuh', 'dah', 'kek', 'ko', 'pls', 'bbrp', 'pd', 'mah', 'dhhh',
    'kpd', 'tuh', 'kzl', 'byar', 'si', 'sii', 'cm', 'sy', 'hahahaha', 'weh',
    'dlu', 'tuhh'
]

data = stop_factory + more_stopwords
dictionary = ArrayDictionary(data)
stp = StopWordRemover(dictionary)

df["Text_wo_stop"] = df["Text_lower"].apply(lambda text: stp.remove(text))
df

,comment,comment_cl2,comment_cl3,Text_lower,Text_wo_stop
0,"""Jika nanti kita bertemu kembali dan setiap ka...",Jika nanti kita bertemu kembali dan setiap kat...,Jika nanti kita bertemu kembali dan setiap kat...,jika nanti kita bertemu kembali dan setiap kat...,nanti bertemu kata keluar mulutku terdengar bi...
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo semoga menjadi pengga...,Sehat selalu Pak Prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...
2,Mimpi,Mimpi,Mimpi,mimpi,mimpi
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024,Bismillah Anies Presiden RI,bismillah anies presiden ri,bismillah anies presiden ri
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,hamparan kami bapa istirahat aja kasih pa ganj...,hamparan bapa istirahat aja kasih pa ganjar pr...
...,...,...,...,...,...
206680,Ganjar dan Prabowo 😎🤗😇....Amin🙏,Ganjar dan Prabowo Amin,Ganjar dan Prabowo Amin,ganjar dan prabowo amin,ganjar prabowo amin
206681,Di facebook juga tempo dot com yg saya baca be...,Di facebook juga tempo dot com yg saya baca be...,Di facebook juga tempo dot com yg saya baca be...,di facebook juga tempo dot com yg saya baca be...,facebook tempo dot com saya baca beritanya itu
206682,@tempodotco mau jadi media seperti lampu merah?,mau jadi media seperti lampu merah,mau jadi media seperti lampu merah,mau jadi media seperti lampu merah,mau jadi media lampu merah
206683,Kalang kabut kalau pak Anis jd presiden 🤣🤣🤣,Kalang kabut kalau pak Anis jd presiden,Kalang kabut kalau pak Anis jd presiden,kalang kabut kalau pak anis jd presiden,kalang kabut kalau pak anis presiden


##### Tokenization

In [12]:
import string
from nltk.tokenize import TweetTokenizer

# Tokenization
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    return tweet_tokens

# Apply tokenisasi
df['Tokenization'] = df['Text_wo_stop'].apply(lambda x: tokenize_tweets(x))
df.head(5)

,comment,comment_cl2,comment_cl3,Text_lower,Text_wo_stop,Tokenization
0,"""Jika nanti kita bertemu kembali dan setiap ka...",Jika nanti kita bertemu kembali dan setiap kat...,Jika nanti kita bertemu kembali dan setiap kat...,jika nanti kita bertemu kembali dan setiap kat...,nanti bertemu kata keluar mulutku terdengar bi...,"[nanti, bertemu, kata, keluar, mulutku, terden..."
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo semoga menjadi pengga...,Sehat selalu Pak Prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,"[sehat, selalu, pak, prabowo, semoga, menjadi,..."
2,Mimpi,Mimpi,Mimpi,mimpi,mimpi,[mimpi]
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024,Bismillah Anies Presiden RI,bismillah anies presiden ri,bismillah anies presiden ri,"[bismillah, anies, presiden, ri]"
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,hamparan kami bapa istirahat aja kasih pa ganj...,hamparan bapa istirahat aja kasih pa ganjar pr...,"[hamparan, bapa, istirahat, aja, kasih, pa, ga..."


##### Stemming 

In [13]:
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# happy emoticon
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'X-p', 'Xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
])
# sad emoticon
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
])
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)


# Stemming
def stem_tweets(tweet_tokens):
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in data and # remove stopwords
              word not in emoticons and # remove emoticons
              word not in string.punctuation): # remove punctuation
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
    return tweets_clean

# Apply stemming
# df.drop(["comment_cl2"], axis=1, inplace=True)
df['stemming'] = df['Tokenization'].apply(lambda x: stem_tweets(x))

# Show the resulting DataFrame
df.head()

,comment,comment_cl2,comment_cl3,Text_lower,Text_wo_stop,Tokenization,stemming
0,"""Jika nanti kita bertemu kembali dan setiap ka...",Jika nanti kita bertemu kembali dan setiap kat...,Jika nanti kita bertemu kembali dan setiap kat...,jika nanti kita bertemu kembali dan setiap kat...,nanti bertemu kata keluar mulutku terdengar bi...,"[nanti, bertemu, kata, keluar, mulutku, terden...","[temu, kata, keluar, mulut, dengar, bijak, smp..."
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo semoga menjadi pengga...,Sehat selalu Pak Prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,"[sehat, selalu, pak, prabowo, semoga, menjadi,...","[sehat, selalu, pak, prabowo, moga, jadi, gant..."
2,Mimpi,Mimpi,Mimpi,mimpi,mimpi,[mimpi],[mimpi]
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024,Bismillah Anies Presiden RI,bismillah anies presiden ri,bismillah anies presiden ri,"[bismillah, anies, presiden, ri]","[bismillah, anies, presiden, ri]"
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,hamparan kami bapa istirahat aja kasih pa ganj...,hamparan bapa istirahat aja kasih pa ganjar pr...,"[hamparan, bapa, istirahat, aja, kasih, pa, ga...","[hampar, bapa, istirahat, aja, kasih, pa, ganj..."


##### Remove Punctuation

In [14]:
# remove punct
def remove_punct(text):
    text = " ".join([char for char in text if char not in string.punctuation])
    return text
df['comments'] = df['stemming'].apply(lambda x: remove_punct(x))
df.head()

,comment,comment_cl2,comment_cl3,Text_lower,Text_wo_stop,Tokenization,stemming,comments
0,"""Jika nanti kita bertemu kembali dan setiap ka...",Jika nanti kita bertemu kembali dan setiap kat...,Jika nanti kita bertemu kembali dan setiap kat...,jika nanti kita bertemu kembali dan setiap kat...,nanti bertemu kata keluar mulutku terdengar bi...,"[nanti, bertemu, kata, keluar, mulutku, terden...","[temu, kata, keluar, mulut, dengar, bijak, smp...",temu kata keluar mulut dengar bijak smp teling...
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo semoga menjadi pengga...,Sehat selalu Pak Prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,"[sehat, selalu, pak, prabowo, semoga, menjadi,...","[sehat, selalu, pak, prabowo, moga, jadi, gant...",sehat selalu pak prabowo moga jadi ganti pak j...
2,Mimpi,Mimpi,Mimpi,mimpi,mimpi,[mimpi],[mimpi],mimpi
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024,Bismillah Anies Presiden RI,bismillah anies presiden ri,bismillah anies presiden ri,"[bismillah, anies, presiden, ri]","[bismillah, anies, presiden, ri]",bismillah anies presiden ri
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,hamparan kami bapa istirahat aja kasih pa ganj...,hamparan bapa istirahat aja kasih pa ganjar pr...,"[hamparan, bapa, istirahat, aja, kasih, pa, ga...","[hampar, bapa, istirahat, aja, kasih, pa, ganj...",hampar bapa istirahat aja kasih pa ganjar pran...


In [15]:
# remove comment kosong
df = df[df['comments']!='']

# reset index
df = df.reset_index(drop=True)
df.head()

,comment,comment_cl2,comment_cl3,Text_lower,Text_wo_stop,Tokenization,stemming,comments
0,"""Jika nanti kita bertemu kembali dan setiap ka...",Jika nanti kita bertemu kembali dan setiap kat...,Jika nanti kita bertemu kembali dan setiap kat...,jika nanti kita bertemu kembali dan setiap kat...,nanti bertemu kata keluar mulutku terdengar bi...,"[nanti, bertemu, kata, keluar, mulutku, terden...","[temu, kata, keluar, mulut, dengar, bijak, smp...",temu kata keluar mulut dengar bijak smp teling...
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,Sehat selalu Pak Prabowo semoga menjadi pengga...,Sehat selalu Pak Prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,sehat selalu pak prabowo semoga menjadi pengga...,"[sehat, selalu, pak, prabowo, semoga, menjadi,...","[sehat, selalu, pak, prabowo, moga, jadi, gant...",sehat selalu pak prabowo moga jadi ganti pak j...
2,Mimpi,Mimpi,Mimpi,mimpi,mimpi,[mimpi],[mimpi],mimpi
3,Bismillah Anies Presiden RI 2024..,Bismillah Anies Presiden RI 2024,Bismillah Anies Presiden RI,bismillah anies presiden ri,bismillah anies presiden ri,"[bismillah, anies, presiden, ri]","[bismillah, anies, presiden, ri]",bismillah anies presiden ri
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,Hamparan kami bapa istirahat aja Kasih pa Ganj...,hamparan kami bapa istirahat aja kasih pa ganj...,hamparan bapa istirahat aja kasih pa ganjar pr...,"[hamparan, bapa, istirahat, aja, kasih, pa, ga...","[hampar, bapa, istirahat, aja, kasih, pa, ganj...",hampar bapa istirahat aja kasih pa ganjar pran...


In [16]:
# remove kolom
df.drop(df.columns[[1,2,3,4,5]], axis = 1, inplace = True)
df.head()

,comment,stemming,comments
0,"""Jika nanti kita bertemu kembali dan setiap ka...","[temu, kata, keluar, mulut, dengar, bijak, smp...",temu kata keluar mulut dengar bijak smp teling...
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,"[sehat, selalu, pak, prabowo, moga, jadi, gant...",sehat selalu pak prabowo moga jadi ganti pak j...
2,Mimpi,[mimpi],mimpi
3,Bismillah Anies Presiden RI 2024..,"[bismillah, anies, presiden, ri]",bismillah anies presiden ri
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,"[hampar, bapa, istirahat, aja, kasih, pa, ganj...",hampar bapa istirahat aja kasih pa ganjar pran...


In [17]:
# save clean data to file
df.to_csv('E:/Documents/Semester 8/TAmbun/Sidang/DBSCAN/Prabowo/DataBersihPrabowo.csv', encoding='utf8', index=False)

### Clustering DBSCAN

In [18]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score

In [19]:
def load_lexicon_lexis(file_path):
    words = []
    scores = []
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split(' ')
            if len(data) >= 2:
                word = data[0].strip()
                score = int(data[1].strip())
                words.append(word)
                scores.append(score)
    return dict(zip(words, scores))

In [20]:
# memuat kamus sentimen
lexicon = load_lexicon_lexis('E:/Documents/Semester 8/TAmbun/Sidang/DBSCAN/Prabowo/kamus.txt')

In [21]:
df = pd.read_csv('E:/Documents/Semester 8/TAmbun/Sidang/DBSCAN/Prabowo/DataBersihPrabowo.csv') 
df.head()

,comment,stemming,comments
0,"""Jika nanti kita bertemu kembali dan setiap ka...","['temu', 'kata', 'keluar', 'mulut', 'dengar', ...",temu kata keluar mulut dengar bijak smp teling...
1,Sehat selalu Pak Prabowo ..semoga menjadi peng...,"['sehat', 'selalu', 'pak', 'prabowo', 'moga', ...",sehat selalu pak prabowo moga jadi ganti pak j...
2,Mimpi,['mimpi'],mimpi
3,Bismillah Anies Presiden RI 2024..,"['bismillah', 'anies', 'presiden', 'ri']",bismillah anies presiden ri
4,Hamparan kami bapa istirahat.aja Kasih pa Ganj...,"['hampar', 'bapa', 'istirahat', 'aja', 'kasih'...",hampar bapa istirahat aja kasih pa ganjar pran...


In [22]:
# tambahkan kolom untuk menyimpan skor sentiment
df['sentiment_score'] = 0

In [23]:
df['comments'] = df['comments'].fillna('').astype(str) # mengganti nilai NaN dan mengubah tipe data menjadi string
for i, row in df.iterrows():
    words = row['comments'].split()
    score = sum([lexicon.get(word, 0) for word in words])
    df.at[i, 'sentiment_score'] = score

In [24]:
# transformasi tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1,2))
X = tfidf_vectorizer.fit_transform(df['comments'])

In [25]:
display(X)

<183998x166151 sparse matrix of type '<class 'numpy.float64'>'
	with 2430641 stored elements in Compressed Sparse Row format>

#### Clustering

In [32]:
# menentukan parameter DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=9, metric='euclidean')

In [33]:
# mendapatkan label kelompok untuk setiap data
labels = dbscan.fit_predict(X)

In [34]:
# mendapatkan jumlah kelompok yang terbentuk
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print("Jumlah kelompok: ", n_clusters)

Jumlah kelompok:  50314


In [35]:
# mengubah data hasil clustering ke dalam format DataFrame
df_clustered = pd.DataFrame({'comments': df['comments'], 'cluster': labels})

##### Evaluation With Silhouette Coefficient


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs

In [ ]:
# generate sample data
X, y = make_blobs(n_samples=183998, centers=3, random_state=42)

In [ ]:
# menghitung silhouette score
score = silhouette_score(X, labels)
print("Silhouette score:", score)

In [ ]:
#menampilkan hasil clustering
df['cluster'] = labels

# Display the number of data points in each cluster
print(df.groupby(['cluster']).count())

In [ ]:
import seaborn as sns

# membuat DataFrame untuk data dan label kelompok
df_cluster = pd.DataFrame(X, columns=['x1', 'x2'])
df_cluster['cluster'] = labels

# membuat plot menggunakan seaborn
sns.scatterplot(x='x1', y='x2', data=df_cluster, hue='cluster', palette='bright')
plt.title(f"Clustering hasil DBSCAN dengan {n_clusters} kelompok")
plt.show()

##### Ekstrak 10 Top Kata 

In [ ]:
import numpy as np

# Menghitung jarak rata-rata dari setiap titik dalam kluster ke titik referensi dalam kluster tersebut
def cluster_center_distance(X, labels, cluster_id):
    cluster_points = X[labels == cluster_id]
    center = np.mean(cluster_points, axis=0)
    distances = np.linalg.norm(cluster_points - center, axis=1)
    return distances

# Menghitung jarak antara setiap dokumen dan pusat kluster dalam konteks DBSCAN
distances = np.zeros((X.shape[0],)) # Inisialisasi matriks jarak
for cluster_id in range(n_clusters):
    cluster_distances = cluster_center_distance(X, labels, cluster_id)
    distances[labels == cluster_id] = cluster_distances

# Untuk setiap cluster yang terbentuk, ambil 10 kata teratas dan identifikasi nilai sentimennya
for cluster_id in range(n_clusters):
    cluster_words = []
    for doc_id, label in enumerate(labels):
        if label == cluster_id:
            words = df.iloc[doc_id]['comments'].split()
            cluster_words.extend(words)
    top_words = pd.Series(cluster_words).value_counts().head(10)
    top_words_scores = [lexicon.get(word, 0) for word in top_words.index]
    print(f"Cluster {cluster_id} Top 10 words: {', '.join(top_words.index)}")
    print(f"Sentiment Scores: {', '.join(map(str, top_words_scores))}")


##### Menghitung Polarity Dari Setiap Cluster

In [ ]:
# hitung rata-rata polarity dari setiap cluster
cluster_sentiment = []
for i in range(n_clusters):
    cluster = df[df['cluster'] == i]
    sentiment_sum = cluster['sentiment_score'].sum()
    num_docs = len(cluster)
    avg_sentiment = sentiment_sum / num_docs
    cluster_sentiment.append(avg_sentiment)
    print("Cluster ", i, " avg sentiment: ", avg_sentiment)


##### Menampilkan Hasil Sentimen

In [ ]:
# Membuat array sentimen posifif, netral, dan negatif
pos_sentiments = np.arange(1, 6)
neu_sentiments = np.array([0])
neg_sentiments = np.arange(-5, 0)

# Membuat fungsi untuk menghitung jumlah sentimen pada setiap cluster
def count_sentiments(cluster):
    pos_count = len([score for score in cluster if score in pos_sentiments])
    neu_count = len([score for score in cluster if score in neu_sentiments])
    neg_count = len([score for score in cluster if score in neg_sentiments])
    return pos_count, neu_count, neg_count

# Membuat list untuk menyimpan jumlah sentimen positif, netral, dan negatif pada setiap cluster
sentiments = []

for i in range(n_clusters):
    cluster = df[df['cluster'] == i]['sentiment_score']
    pos_count, neu_count, neg_count = count_sentiments(cluster)
    sentiments.append((pos_count, neu_count, neg_count))

# Menampilkan jumlah sentimen positif, netral, dan negatif pada setiap cluster
for i, sentiment in enumerate(sentiments):
    print(f"Cluster {i}: {sentiment[0]} sentimen positif, {sentiment[1]} sentimen netral, {sentiment[2]} sentimen negatif")


In [ ]:
# Mengakumulasi total jumlah sentimen positif, negatif, dan netral dari keseluruhan clusternya
total_pos_count = sum([sentiment[0] for sentiment in sentiments])
total_neu_count = sum([sentiment[1] for sentiment in sentiments])
total_neg_count = sum([sentiment[2] for sentiment in sentiments])

# Menampilkan total jumlah sentimen positif, netral, dan negatif dari keseluruhan clusternya
print(f"Total sentimen positif: {total_pos_count}")
print(f"Total sentimen netral: {total_neu_count}")
print(f"Total sentimen negatif: {total_neg_count}")


In [ ]:
import matplotlib.pyplot as plt

# Menyiapkan data untuk pie chart
labels = ['Positif', 'Netral', 'Negatif']
sizes = [total_pos_count, total_neu_count, total_neg_count]
colors = ['#1f77b4', '#ff7f0e', '#d62728']

# Membuat pie chart
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)

# Menambahkan judul pada pie chart
ax.set_title('Total Sentimen pada Seluruh Cluster')

# Menampilkan pie chart
plt.show()
